https://github.com/AbdulDridi/Knowledge-Based-WSD
이 소스는 위의 github주소에 기반합니다.


In [1]:
!pip3 uninstall numpy
!pip3 install folium==0.2.1
!pip3 install imgaug==0.2.5
!pip3 install --upgrade numpy==1.16.1
!pip3 freeze ## 파이썬 패키지 설치된거 버전 몇인지 보여줌

Uninstalling numpy-1.16.4:
  Would remove:
    /usr/bin/f2py
    /usr/bin/f2py3
    /usr/bin/f2py3.6
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.6
    /usr/local/lib/python3.6/dist-packages/numpy-1.16.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.16.4
     |████████████████████████████████| 71kB 2.9MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79980 sha256=24806d38ed7889cda65e62dcb670bd01a5cc5a63030c3dcee6a6b8a834603f98
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 563kB 2.9MB/s 
     |████████████████████████████████| 20.4MB 43.2MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp36-none-any.whl size=56144

     |████████████████████████████████| 17.3MB 2.8MB/s 
  Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0


absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.1.0
astor==0.8.0
astropy==3.0.5
atari-py==0.1.15
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.8
autograd==1.2
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.9.189
botocore==1.12.189
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
certifi==2019.6.16
cffi==1.12.3
chainer==5.4.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cvxopt==1.2.3
cvxpy==1.0.15
cycler==0.10.0
cymem==2.0.2
Cython==0.29.12
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.0
distributed==1.25.3
Django==2.2.3
dlib==19.16.0
dm-sonnet==1.34
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
easy

***WordNet내 모든 Graph 정보 추출***

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import numpy as np


tokenizer = RegexpTokenizer(r'\w+')
stopWords = set(stopwords.words('english'))

# Synset 세부정보 추출 관련 class
class SynsetDetails:

    def __init__(self, synset):
        self.synset = synset
        self.definitionSynsets = self.getDefinitionSynsets(
                                            self.getFilteredTokenizedDefinition(
                                                tokenizer.tokenize(synset.definition())))

    # 불용어 처리 definition
    def getFilteredTokenizedDefinition(self, definition):
        filteredTokenizedDefinition = []
        for word in definition:
            if word not in stopWords:
                filteredTokenizedDefinition.append(word)
        return filteredTokenizedDefinition

    # synset definition 추출
    def getDefinitionSynsets(self, filteredTokenisedDefinition):
        definitionSynsets = dict()
        for word in filteredTokenisedDefinition:
            wordsSynsets = wordnet.synsets(word)
            if wordsSynsets != []:
                definitionSynsets[word] = wordsSynsets
        return definitionSynsets

# Synset이 가지고 있는 모든 relation 추출..
def getWordnetSynsetRelations():
    allSynsets = []
    count = 0
    synsetIndex = dict()
    for synset in wordnet.all_synsets():
        currentSynsetDetails = SynsetDetails(synset)
        allSynsets.append(currentSynsetDetails)
        synsetIndex[(synset.pos(),synset.offset())] = count
        count+=1
    return [allSynsets, synsetIndex]

# matrix 형태로 graph 생성.
def generateAndSaveWordnetGraph():
    wordnetSynsetRelations = getWordnetSynsetRelations()
    wordnetSynsetIndexes = wordnetSynsetRelations[1]
    row = []
    col = []
    data = []
    for synsetEntry in wordnetSynsetRelations[0]:
        # definition에 따라서 관련 syset들 link
        synsetLinks = list(synsetEntry.definitionSynsets.values())
        # 상위어 
        # 하위어 
        # 전체어 = 단어 tree가 구성하는 전체어 : forest, organism.
        synsetLinks.append(synsetEntry.synset.hypernyms())
        synsetLinks.append(synsetEntry.synset.hyponyms())
        synsetLinks.append(synsetEntry.synset.member_holonyms())

        # synset link 결합.
        synsetLinksUnion = []
        for listOfSynsets in synsetLinks:
            synsetLinksUnion = synsetLinksUnion + listOfSynsets

        # 중복 제거. 중복된 링크 제거
        synsetLinksUnion = set(synsetLinksUnion)
        for linkedToSynset in synsetLinksUnion:
            col.append(wordnetSynsetIndexes.get((synsetEntry.synset.pos(),synsetEntry.synset.offset())))
            row.append(wordnetSynsetIndexes.get((linkedToSynset.pos(),linkedToSynset.offset())))
            data.append(1)
    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    np.savez("initialGraphData",row=row,col=col,data=data,synsetIndexes=wordnetSynsetIndexes)
    return None
generateAndSaveWordnetGraph()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***KGB + PPR Algorithm***

In [3]:
from nltk.corpus import wordnet
import numpy as np
from nltk.corpus import stopwords
from scipy.sparse import csc_matrix
from sklearn.preprocessing import normalize

import nltk
nltk.download('wordnet')
nltk.download('stopwords')

# 그래프 정보 불러오기
def makeWordnetGraphAndLoadSynsetIndexDict():
    graphDataFile = np.load("./initialGraphData.npz",allow_pickle=True)
    synsetIndexes = graphDataFile['synsetIndexes'].item()
    length = len(synsetIndexes.keys())
    data = graphDataFile['data']
    row = graphDataFile['row']
    col = graphDataFile['col']
    wordnetAdjacencyMatrixGraph = csc_matrix((data, (row, col)),
                                            shape=(length, length))
    return (wordnetAdjacencyMatrixGraph, synsetIndexes)

# 초기값 설정
def getInitialRankDistribution(textSynsets, noOfNodesInGraph, synsetIndexes):
    if(len(textSynsets)!= 0):
        initialRankDistribution = np.zeros(noOfNodesInGraph)
        rankSplitBetweenWords = 1/len(textSynsets)
        rankPerSynset = 0
        for synsetList in textSynsets:
            rankPerSynset = rankSplitBetweenWords / (len(synsetList))
            for synset in synsetList:
                pos = synset.pos()
                offset = synset.offset()
                nodeIndex = synsetIndexes.get((pos,offset))
                initialRankDistribution[nodeIndex] += rankPerSynset
    else:
        initialRankDistribution = np.full(noOfNodesInGraph, 1/noOfNodesInGraph)
    return initialRankDistribution

# 전체 그래프에 대한 Page Rank 계산
# 문장에 등장한 단어에 대해서 Personalized 진행
def performPPRAndGetSynsetsFromSynsets(textSynsets,transitionMatrix,
                                        synsetIndexes,dampingFactor,iterations):
    initialRankDistribution = getInitialRankDistribution(textSynsets,
                                transitionMatrix.get_shape()[0],synsetIndexes)
    pageRankVector = initialRankDistribution.copy()
    for i in range(iterations):
        pageRankVector = (dampingFactor
                          * (transitionMatrix.dot(pageRankVector)))\
                          + (1-dampingFactor) * initialRankDistribution

    return getTextSynsets(textSynsets,pageRankVector,synsetIndexes)

# 가장 높은 랭킹의 의미 선택.
def getTextSynsets(textSynsets ,pageRankVector, synsetIndexes):
    disambiguatedSynsets = []
    for wordSynsets in textSynsets:
        currentSynset = None
        currentSynsetRank = 0
        for synset in wordSynsets:
            pos = synset.pos()
            offset = synset.offset()
            synsetPagerankValue = pageRankVector[
                                                synsetIndexes.get((pos,offset))]
            if synsetPagerankValue > currentSynsetRank:
                currentSynset = synset
                currentSynsetRank = synsetPagerankValue
        disambiguatedSynsets.append(currentSynset)
    return disambiguatedSynsets


def makeTransitionMatrix(adjacencyMatrix):
    wordnetTransitionMatrix = csc_matrix.copy(adjacencyMatrix)
    wordnetTransitionMatrix = normalize(wordnetTransitionMatrix,
                                        norm='l1', axis=1)
    return wordnetTransitionMatrix

# 입력 문장의 단어들의 synset 반환.
def getSetSynsets(text):
    tokenizedText = text
    stopWords = set(stopwords.words('english'))
    stopWords.add("I")
    wordsWithSynsets = list()
    sentence = list()
    for word in tokenizedText:
        synsets = wordnet.synsets(word)
        if synsets != []:
            wordsWithSynsets.append(synsets)
            sentence.append(True)
        else:
            sentence.append(False)
    return (sentence,wordsWithSynsets)


data = makeWordnetGraphAndLoadSynsetIndexDict()
adjacencyMatrix = data[0]
synsetIndexes = data[1]
transitionMatrix = makeTransitionMatrix(adjacencyMatrix)


sentence = ["they", "eat", "a", "meal"]

sentencesSynsets = list()
sentencesMeaningfulSynsets = list()
setOfSynsets = getSetSynsets(sentence)
sentencesSynsets.append(setOfSynsets[0])
sentencesMeaningfulSynsets.append(setOfSynsets[1])


## 페이지링크 돌려서 중의성 선택됨
disambiguatedSentence = performPPRAndGetSynsetsFromSynsets(sentencesMeaningfulSynsets[0],
                                                           transitionMatrix,
                                                           synsetIndexes,
                                                           0.6,
                                                           50)
print(disambiguatedSentence)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[Synset('eat.v.02'), Synset('vitamin_a.n.01'), Synset('meal.n.01')]


In [0]:
## in_a 오류
## wsd 기반 사용할떄 이거 갖다 써라